In [1]:
import cv2
import open3d as o3d
from open3d.web_visualizer import draw
import time
import json
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
def stream_camera(config_filename, number_of_images=1):
    camera_rgbd_images = []

    # Load JSON with configuration
    with open(config_filename) as cf:
        rs_cfg = o3d.t.io.RealSenseSensorConfig(json.load(cf))

    # Initialize device and get metadata
    rscam = o3d.t.io.RealSenseSensor()
    rscam.init_sensor(rs_cfg)
    rgbd_metadata = rscam.get_metadata()

    # Get intrinsic parameters from metadata
    fx = rgbd_metadata.intrinsics.intrinsic_matrix[0][0]
    fy = rgbd_metadata.intrinsics.intrinsic_matrix[1][1]
    ppx = rgbd_metadata.intrinsics.intrinsic_matrix[0][2]
    ppy = rgbd_metadata.intrinsics.intrinsic_matrix[1][2]
    height = rgbd_metadata.height
    width = rgbd_metadata.width



    # Start acquiring images
    rscam.start_capture(start_record=True)
    print('Waiting 2 seconds for camera to start...')
    time.sleep(2)
    print('Capturing frames')
    #mask_depth = cv2.imread("/home/avena/PycharmProjects/tsdf_gpu/input/01_mask_c1_16u.png", cv2.IMREAD_UNCHANGED)
    #mask_color = cv2.imread("/home/avena/PycharmProjects/tsdf_gpu/input/01_mask_c1_8u.png", cv2.IMREAD_UNCHANGED)
    #mask_color = cv2.cvtColor(mask_color, cv2.COLOR_GRAY2RGB)
    #mask_depth = mask_depth.astype(np.float32)
    #print(mask_color.shape)
    #print(mask_depth.shape)
    for image_idx in range(number_of_images):
        rgbd_frame = rscam.capture_frame(align_depth_to_color=True)
        rgbd_frame = rgbd_frame.to_legacy_rgbd_image()
        depth = np.asarray(rgbd_frame.depth).astype(np.float32)
        color = np.asarray(rgbd_frame.color)
        print(color.shape)
        # Bilateral filter
        # depth_filtered = cv2.bilateralFilter(src=depth,
        #                                      d=3,
        #                                      sigmaColor=2,
        #                                      sigmaSpace=2,
        #                                      borderType=cv2.BORDER_CONSTANT
        #                                      )
        #
        #
        # color = cv2.bitwise_and(color, mask_color)
        # depth_filtered = cv2.bitwise_and(depth_filtered, mask_depth)

        depth_filtered = cv2.ximgproc.jointBilateralFilter(joint=color.astype(np.float32),
                                                           src=depth,
                                                           d=3,
                                                           sigmaColor=2,
                                                           sigmaSpace=2,
                                                           borderType=cv2.BORDER_CONSTANT).astype(np.uint16)
        # depth_filtered = depth

        color_tensor = o3d.core.Tensor(color, dtype=o3d.core.Dtype.UInt8)
        depth_tensor = o3d.core.Tensor(depth_filtered, dtype=o3d.core.Dtype.UInt16)
        color_image = o3d.t.geometry.Image(color_tensor)
        depth_image = o3d.t.geometry.Image(depth_tensor)
        rgbd_frame = o3d.t.geometry.RGBDImage(color_image, depth_image)

        camera_rgbd_images.append(rgbd_frame)

    # Stop device
    rscam.stop_capture()

    # Return results
    return camera_rgbd_images, {"fx": fx, "fy": fy, "cx": ppx, "cy": ppy, "height": height, "width": width}

In [3]:
def tsdf_volume(frames, camera_info, device):
    # TSDF volume initialization
    volume = o3d.t.geometry.TSDFVoxelGrid(
        map_attrs_to_dtypes={
            'tsdf': o3d.core.Dtype.Float32,
            'weight': o3d.core.Dtype.UInt16,
            'color': o3d.core.Dtype.UInt16
        },
        voxel_size=0.002,
        sdf_trunc=0.005,
        block_resolution=16,
        block_count=50000,
        device=device,
    )

    # Same camera, so transform is identity
    extrinsic = np.eye(4)
    extrinsic_gpu = o3d.core.Tensor(extrinsic, o3d.core.Dtype.Float32, device)
    intrinsic = o3d.camera.PinholeCameraIntrinsic(width=camera_info["width"],
                                                  height=camera_info["height"],
                                                  fx=camera_info["fx"], fy=camera_info["fy"],
                                                  cx=camera_info["cx"], cy=camera_info["cy"])
    intrinsic_gpu = o3d.core.Tensor(intrinsic.intrinsic_matrix, o3d.core.Dtype.Float32, device)

    # Process each frame
    start = time.perf_counter()
    for frame in frames:
        color_gpu = frame.color.to(device)
        depth_gpu = frame.depth.to(device)
        volume.integrate(depth=depth_gpu,
                         color=color_gpu,
                         intrinsics=intrinsic_gpu,
                         extrinsics=extrinsic_gpu,
                         depth_scale=1000.0,
                         depth_max=1.2,
                         )

    end = time.perf_counter()
    print('Mean compute time:', (end - start) / len(frames) * 1000, ' [ms]')

    # Extract point cloud from volume
    return_value = volume.cpu().extract_surface_points()

    # But not sure whether this works or not...
    o3d.core.cuda.release_cache()
    return return_value

In [4]:
def get_filtered_pointcloud(camera_config_path, device, frames_no):
    frames, camera_info = stream_camera(camera_config_path, frames_no)
    filtered_cloud = tsdf_volume(frames, camera_info, device)
    return filtered_cloud

In [5]:
frames_no = 10

print('Processing using GPU')
device = o3d.core.Device("CUDA:0")
pcd_gpu = get_filtered_pointcloud(camera_config_path="input/cam7_config.json",
                                  device=device,
                                  frames_no=frames_no
                                  )

# print('Processing using CPU')
# device = o3d.core.Device("CPU:0")
# pcd_cpu = get_filtered_pointcloud(camera_config_path="/home/avena/PycharmProjects/tsdf/input/cam5_config.json",
#                                   device=device,
#                                   frames_no=frames_no
#                                   )
#
# o3d.visualization.draw([pcd_gpu, pcd_cpu])
draw([pcd_gpu])

# o3d.io.write_point_cloud("output/depth_bilateral_filter_gpu.ply", pcd_gpu.to_legacy())

Processing using GPU
Waiting 2 seconds for camera to start...[Open3D INFO] Capture started with RealSense camera 135222250712

Capturing frames
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
[Open3D INFO] Capture stopped.
Mean compute time: 1.7849533999651612  [ms]
[Open3D WARNING] No estimated max point cloud size provided, using a 2-pass estimation. Surface extraction could be slow.
[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: {"stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"}
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


Failed to establish dbus connection

WebVisualizer(window_uid='window_0')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate


In [6]:
def bilateral_filter(img, d, sigma_color, sigma_space):
    return cv2.bilateralFilter(img, d, sigma_color, sigma_space)

In [7]:
def joint_bilateral_filter(color, depth, d, sigmaColor, sigmaSpace, borderType):
    return cv2.ximgproc.jointBilateralFilter(joint=color,
                                            src=depth,
                                            d=d,
                                            sigmaColor=sigmaColor,
                                            sigmaSpace=sigmaSpace,
                                            borderType=borderType)

In [8]:
def guided_filter(color, depth, radius, eps):
    return cv2.ximgproc.guidedFilter(guide=color,
                                    src=depth,
                                    radius=radius,
                                    eps=eps)

In [9]:
def radius_outlier_removal(point_cloud, nb_neighbors, std_ratio):
    point_cloud = point_cloud.to_legacy_pointcloud()
    _, ind = point_cloud.remove_statistical_outlier(nb_neighbors, std_ratio)
    point_cloud = point_cloud.select_by_index(ind)
    point_cloud_gpu = o3d.t.geometry.PointCloud.from_legacy_pointcloud(point_cloud)
    return point_cloud_gpu

In [10]:
def statistical_outlier_removal(point_cloud, nb_points, radius):
    point_cloud = point_cloud.to_legacy_pointcloud()
    _, ind = point_cloud.remove_radius_outlier(nb_points, radius)
    point_cloud = point_cloud.select_by_index(ind)
    point_cloud_gpu = o3d.t.geometry.PointCloud.from_legacy_pointcloud(point_cloud)
    return point_cloud_gpu

In [11]:
draw(radius_outlier_removal(pcd_gpu, 20, 2.0))

FEngine (64 bits) created at 0x7f00280064d0 (threading is enabled)
EGL(1.5)
OpenGL(4.1)
[Open3D INFO] Window window_1 created.


WebVisualizer(window_uid='window_1')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.5134082588113147
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.5134082588113147
[Open3D INFO] Sending init frames to window_0.
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate


In [15]:
pcd = radius_outlier_removal(pcd_gpu, 20, 2.0)
pcd = pcd.to_legacy_pointcloud()
o3d.io.write_point_cloud("nazwa.ply",pcd)

True

In [12]:
draw(statistical_outlier_removal(pcd_gpu, 16, 0.05))

[Open3D INFO] Window window_2 created.


WebVisualizer(window_uid='window_2')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.01151207486524397
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.01151207486524397
[Open3D INFO] Sending init frames to window_1.
[Op

[000:000][63062] (stun_port.cc:96): Binding request timed out from 10.3.15.x:58964 (enp4s0)
[000:005][63062] (stun_port.cc:96): Binding request timed out from 10.3.15.x:36957 (enp4s0)
[003:208][63062] (stun_port.cc:96): Binding request timed out from 10.3.15.x:59859 (enp4s0)


In [13]:
cv2.bitwise_and

<function bitwise_and>